Straight forward method to extract relevant details using string pattern matching
Its easy to implement, but need proper string pattern mapping for this to work thus its less robust solution for the problem.

In [16]:
import re
from pdfminer.high_level import extract_pages, extract_text
import pdfplumber

def extract_details_from_pdf(pdf_path):
    # Extract text from the PDF
    text = extract_text(pdf_path)

    # Define a dictionary to store the bank details
    bank_details = {}

    # Use regular expressions to find the relevant information
    account_pattern = r"Account #:\s*(\d+)"
    ifsc_pattern = r"IFSC Code:\s*([A-Z0-9]+)"
    bank_pattern = r"Bank:\s*([A-Za-z\s]+)"
    branch_pattern = r"Branch:\s*([A-Za-z\s-]+)"

    # Search for the patterns in the extracted text
    account_match = re.search(account_pattern, text)
    ifsc_match = re.search(ifsc_pattern, text)
    bank_match = re.search(bank_pattern, text)
    branch_match = re.search(branch_pattern, text)

    # Store the matched groups in the dictionary
    if account_match:
        bank_details['Account'] = account_match.group(1)
    if ifsc_match:
        bank_details['IFSC'] = ifsc_match.group(1)
    if bank_match:
        bank_details['Bank'] = bank_match.group(1).split("\n")[0]
    if branch_match:
        bank_details['Branch'] = branch_match.group(1).split("\n")[0]

    invoice_details = {}
    gstin_pattern = r"GSTIN\s*([\dA-Z]+)"
    address_pattern = r"C/o\s*([\w\s,.-]+)"
    mobile_pattern = r"Mobile\s*\+?(\d{2}\s*\d{10})"
    email_pattern = r"Email\s*([\w\.-]+@[\w\.-]+)"
    invoice_number_pattern = r"Invoice #:\s*(\w+-\d+)"
    invoice_date_pattern = r"Invoice Date:\s*([\d\s\w]+)"
    due_date_pattern = r"Due Date:\s*([\d\s\w]+)"
    customer_pattern = r"Customer Details:\s*([\w\s]+)"
    place_of_supply_pattern = r"Place of Supply:\s*([\dA-Z\s-]+)"

    # Search for the patterns in the extracted text
    gstin_match = re.search(gstin_pattern, text)
    address_match = re.search(address_pattern, text)
    mobile_match = re.search(mobile_pattern, text)
    email_match = re.search(email_pattern, text)
    invoice_number_match = re.search(invoice_number_pattern, text)
    invoice_date_match = re.search(invoice_date_pattern, text)
    due_date_match = re.search(due_date_pattern, text)
    customer_match = re.search(customer_pattern, text)
    place_of_supply_match = re.search(place_of_supply_pattern, text)

    # Store the matched groups in the dictionary
    if gstin_match:
        invoice_details['GSTIN'] = gstin_match.group(1)
    if address_match:
        invoice_details['Address'] = ",".join(address_match.group(1).strip().split("\n")[:-1])
    if mobile_match:
        invoice_details['Mobile'] = mobile_match.group(1).split()[1]
    if email_match:
        invoice_details['Email'] = email_match.group(1)
    if invoice_number_match:
        invoice_details['Invoice Number'] = invoice_number_match.group(1)
    if invoice_date_match:
        invoice_details['Invoice Date'] = invoice_date_match.group(1).split("\n")[0]
    if due_date_match:
        invoice_details['Due Date'] = due_date_match.group(1).split("\n")[0]
    if customer_match:
        invoice_details['Customer'] = customer_match.group(1).strip().split("\n")[0]
    if place_of_supply_match:
        invoice_details['Place of Supply'] = place_of_supply_match.group(1).strip()

    #item_details stored in tables
    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf.pages)):
            page = pdf.pages[page_num]
            
            # Extract tables from the page
            tables = page.extract_tables()
            item_details = tables[0]

    return bank_details,invoice_details,item_details

# Example usage
pdf_path = 'sample.pdf'
bank_details ,invoice_details,item_details= extract_details_from_pdf(pdf_path)
print(invoice_details)
for row in item_details:
    print(row)
print(bank_details)


{'GSTIN': '23AADCU2395N1ZY', 'Address': 'KARUNA GUPTA KURELE, 1st Floor,S.P Bungalow Ke Pichhe, Shoagpur Shahdol, Shahdol,Shahdol, MADHYA PRADESH, 484001', 'Mobile': '8585960963', 'Email': 'ruhi@dermaq.in', 'Invoice Number': 'INV-73', 'Invoice Date': '15 Feb 2024', 'Due Date': '26 Oct 2023', 'Customer': 'Avik Mallick', 'Place of Supply': '23-MADHYA PRADESH'}
['Taxable Amount ₹5,374.02']
['CGST 6.0% ₹186.70']
['SGST 6.0% ₹186.70']
['CGST 9.0% ₹21.75']
['SGST 9.0% ₹21.75']
['Round Off 0.09']
['Total ₹5,791.00']
['Total Discount ₹3,796.34']
{'Account': '1146860541', 'Bank': 'Kotak Mahindra Bank', 'Branch': 'PUNE - CHINCHWAD'}
